# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Principal Component Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd
pd.options.mode.chained_assignment = None  # default='warn'


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_500Cases_H2_NoNoise/')


NVarsRed     = 8

valid_perc   = 20.

FixedMinVal  = 1.e-20

n_ics        = 500


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

In [3]:
input_vars_        = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PPC/CleanVars_ToRed.csv', header=None).to_numpy()[0,:]
try:
    input_vars_not = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PPC/CleanVars_NotToRed.csv', header=None).to_numpy()[0,:]
except:
    input_vars_not = []
input_vars         = np.append(input_vars_, input_vars_not)

In [4]:
Vars = list(input_vars_not)
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0    = []
for Var in input_vars_not:
    Vars0.append(Var+'0')
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
InputVar = ['t'] + Vars0

In [5]:
Header           = Vars[0]
for var in Vars[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PPC/Vars.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

Header           = Vars0[0]
for var in Vars0[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PPC/Vars0.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

In [6]:
# FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
# Data         = pd.read_csv(FileName, header=0)
# SimIdxs      = Data.to_numpy(int)


# FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
# Data         = pd.read_csv(FileName)
# ICVecs       = Data.to_numpy()
# n_ics         = len(ICVecs)


# P0Vec        = []
# EqRatio0Vec  = []
# T0Vec        = []
# for iC in range(n_ics):
#     Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
#     P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
#     EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
#     T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
# P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
# EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
# T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(n_ics):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)


for iC in range(n_ics):
    try:
        FileName         = OutputDir+'/'+str(NVarsRed)+'PPC/train/ext/PPC.csv.'+str(iC+1)
        DataNew          = pd.read_csv(FileName, header=0)
        Nts              = len(DataNew)
        DataTemp         = DataNew.iloc[[0]][Vars] #DataTemp.iloc[1][Vars]
        DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
        DataTemp.reset_index(drop=True, inplace=True)
        DataNew[Vars0]   = DataTemp
        DataTemp['t']    = DataNew['t']
        DataNew['t']     = np.maximum(DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy(), FixedMinVal)
        if (iC==0):
            Data         = DataNew
        else:
            Data         = pd.concat([Data, DataNew], axis=0)
    except:
        pass
Data.reset_index(drop=True, inplace=True)

In [7]:
Data

,t,T,H2,OH,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,...,H20,OH0,PC0_1,PC0_2,PC0_3,PC0_4,PC0_5,PC0_6,PC0_7,PC0_8
0,1.000000e-20,3.075912,-1.023633,-30.000000,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,...,-1.023633,-30.0,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,-1.160455,-6.507348
1,1.000000e-14,3.075912,-1.023633,-24.862454,4.623424,4.349837,4.072447,8.107039,-4.190365,15.030663,...,-1.023633,-30.0,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,-1.160455,-6.507348
2,1.000000e-07,3.075912,-1.023633,-11.710813,2.558594,0.820974,-1.074489,0.095924,-3.010083,0.575849,...,-1.023633,-30.0,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,-1.160455,-6.507348
3,1.032962e-07,3.075912,-1.023633,-11.686344,2.551587,0.815362,-1.080723,0.095088,-2.994770,0.548570,...,-1.023633,-30.0,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,-1.160455,-6.507348
4,1.067011e-07,3.075912,-1.023633,-11.661964,2.544563,0.809762,-1.086925,0.094380,-2.979289,0.521410,...,-1.023633,-30.0,5.768444,8.129791,8.962553,13.770906,0.875689,-7.918133,-1.160455,-6.507348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248495,8.783386e-01,3.388650,-1.183726,-2.914539,-0.489996,1.805739,-0.253892,-0.398692,0.771423,0.276287,...,-1.037138,-30.0,5.768421,8.085975,9.067291,13.654582,0.939557,-7.904568,-1.109712,-6.590316
248496,9.072907e-01,3.388650,-1.183726,-2.914539,-0.489996,1.805739,-0.253892,-0.398692,0.771423,0.276287,...,-1.037138,-30.0,5.768421,8.085975,9.067291,13.654582,0.939557,-7.904568,-1.109712,-6.590316
248497,9.371972e-01,3.388650,-1.183726,-2.914539,-0.489996,1.805739,-0.253892,-0.398692,0.771423,0.276287,...,-1.037138,-30.0,5.768421,8.085975,9.067291,13.654582,0.939557,-7.904568,-1.109712,-6.590316
248498,9.680895e-01,3.388650,-1.183726,-2.914539,-0.489996,1.805739,-0.253892,-0.398692,0.771423,0.276287,...,-1.037138,-30.0,5.768421,8.085975,9.067291,13.654582,0.939557,-7.904568,-1.109712,-6.590316


In [8]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/valid/')
except:
    pass

In [9]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Output.csv', index=False)

In [10]:
data_id    = 'res'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/')
except:
    pass

DataInput        = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = DataNoZero[['t'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Output.csv', index=False)

In [11]:
data_id = 'ics'

Data_ics                     = Data[Data['t'] == FixedMinVal]
n_points_ics                 = len(Data_ics)

idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

n_valid                      = len(valid_idx)
n_train                      = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/')
except:
    pass

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Input.csv', index=False)

DataOutput     = Data_ics[['t'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PPC/valid/'+data_id+'/Output.csv', index=False)

In [12]:
# data_id = 'scs'

# Mask                         = [i for i in range(1,len(Data)-1) if Data.iloc[i-51]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]
# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput     = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [13]:
# data_id = 'fcs'

# Mask                         = [i for i in range(-1,len(Data)-1) if Data.iloc[i+1]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]

# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=43)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput       = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)